In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [4]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [155]:
# Example 1
x = [2.0, 3.0, -1.0] # inputs
n = MLP(3, [4, 4, 1]) # MLP(inputs, layers=(4 node layer, 4 node layer, 1 node layer - output))
n(x) # passing the inputs to the Multi Layer Perceptron

Value(data=-0.467580682178557)

In [156]:
# Example 2
xs = [
  [2.0, 3.0, -1.0], # expected result = 1.0
  [3.0, -1.0, 0.5], # expected result = -1.0
  [0.5, 1.0, 1.0], # expected result = -1.0
  [1.0, 1.0, -1.0], # expected result = 1.0
] # list of groups of inputs
ys = [1.0, -1.0, -1.0, 1.0] # desired targets for each group of inputs

ypred = [n(x) for x in xs] # predicted results from MLP for each group of inputs
ypred

[Value(data=-0.467580682178557),
 Value(data=-0.7937318142582119),
 Value(data=-0.334029126970337),
 Value(data=-0.26969308494363764)]

In [157]:
# Calculating Loss value for example 2 - using mean square error function for loss
list_of_losses = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]
# zip ex: a = [1,2,3] b = [a,b,c] zip(a,b) = [[1,a],[2,b],[3,c]]
# ygt means Y ground truth which is the expected result

list_of_losses
# returned a list of losses for each group of inputs

loss = sum(list_of_losses)
# total loss = the sum of all losses of expected results

loss

Value(data=4.251977356830471)

In [158]:
# calculating grade of nodes with derivative 

# because ypred is a list of output nodes of the MLP and we apply - and ** to those nodes which are instances of Value that has a -,** method
# it means that the result of list of losses and loss itself are Value objects

# losses = loss of ys0 + loss of ys1 + loss of ys2 + loss of ys3
# loss of ys0 = object MLP + loss function -> loss function = (ys0 - ygt0)**2 -> object MLP is a graph with the output node ys0

# doing backward() on loss will spread the gradient (derivate) to the loss function but also to the MLP
# due to that we have, 1) how much the nodes affect the loss function, but also, 2) how much the nodes affect the output node ys0

loss.backward() # always do one calculation per MLP creation because derivates add up and can acumulate
n.layers[0].neurons[0].w[0].grad
# how much this affects the loss or the output results
# first layer, first neuron, first weight (one weight per input on a neuron, each neuron has 3 inputs), gradient derivative

# the goal is to get a prediction or results as close as the ygt y ground truth value (expected result) to reduce loss as close to zero we can
# if we make a Weight with a possitive gradient higher we will get a higher output
# if we make a Weight with a negative gradient lower we will get a lower output
# if the output is closer to the expected result we will get a lower loss
# if the output is farther from the expected result we will get a higher loss

-2.297664602657766

In [159]:
# getting all the parameters of the MLP - values we can interact with to change the output (weights and biases)

n.parameters()
# node1w1,node1w2,node1w3,node1bias,node2w1,node2w2,node2w3,node2bias, ... 
# has all the objects inside we can modify them

[Value(data=-0.3583313394887),
 Value(data=-0.30611862768222275),
 Value(data=-0.6088959513388303),
 Value(data=0.35247194902681844),
 Value(data=-0.12697716158392014),
 Value(data=0.5063229217066136),
 Value(data=0.6009473590299907),
 Value(data=-0.21902848976868872),
 Value(data=0.6651874938402131),
 Value(data=0.04134328015762678),
 Value(data=0.02930934542114083),
 Value(data=0.07584833581811146),
 Value(data=-0.7201009178657736),
 Value(data=-0.7782069461436816),
 Value(data=-0.9950470436667604),
 Value(data=-0.29110565245512765),
 Value(data=0.3912326396202157),
 Value(data=0.6233286921256274),
 Value(data=-0.0614045130081331),
 Value(data=0.3819372480198273),
 Value(data=-0.40869117974083813),
 Value(data=0.46776635669121935),
 Value(data=-0.9145900914267369),
 Value(data=-0.4011992047077457),
 Value(data=-0.4857809090020808),
 Value(data=-0.3036490107170904),
 Value(data=-0.46246937649994146),
 Value(data=0.48731652527256886),
 Value(data=0.17173448987386908),
 Value(data=0.168

In [160]:
# Gradient decent

# we modify each parameter by a tiny STEP size in the direction of the gradient

# following the direction can get us closer to the loss
print({"g": n.layers[0].neurons[0].w[0].grad, "w": n.layers[0].neurons[0].w[0].data})
# 1) if grad is negative means that making W bigger the lost will be lower
# 2) if grad is possitive means that making W smaller the lost will be lower

# if the amount to add is (step * grad) means:
# 1) if grad is negative, W must get bigger -> (step(negative) * grad(negative)) -> step must be negative to cancel the grad
# 2) if grad is possitive, W must get smaller -> (step(negative) * grad(possitive)) -> step must be negative to take out the grad

# the previous is "following the direction of the gradient"

{'g': -2.297664602657766, 'w': -0.3583313394887}


In [161]:
# changing the weights
stepSign = -1

# update - stocaistic gradient decent update - just adding the product of grad * step

# no batching - means we do an update in all the data, when the data is too large, batching is a must and we only do an update in a batch
# that can be grabbed randomly

for p in n.parameters():
    p.data += stepSign * 0.01 * p.grad
    
n.layers[0].neurons[0].w[0].data
# new weight

-0.3353546934621223

In [162]:
ypred = [n(x) for x in xs] # predicted results from MLP for each group of inputs
print(ypred)
# Calculating Loss after changing weights
new_loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
print({"old loss": loss, "new loss": new_loss})
loss = new_loss
loss.backward() # backward pass -> recalculate gradient

[Value(data=-0.24400936649583502), Value(data=-0.739663299092658), Value(data=-0.21871892658341632), Value(data=-0.025416361134800725)]
{'old loss': Value(data=4.251977356830471), 'new loss': Value(data=3.2772133311305924)}


In [163]:
# training the Model multiple times

iterations_training = 200 # iterations_training

print("Expected result:\n")
print(ys)
print("---------------------------------------\n")
for k in range(iterations_training):
    # Forward pass
    ypred = [n(x) for x in xs] # predicted results from MLP for each group of inputs
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
    # zero grad to flush previous gradients
    for p in n.parameters():
        p.grad = 0.0
    # Backward pass
    loss.backward()
    # Update
    for p in n.parameters():
        p.data += -0.05 * p.grad
    # Show status
    print(ypred)
    print(k, {"loss": loss.data})
    
# if the step is too large and goes over the point where loss is close to zero, the loss will change values into something that will not make sense
# to get closer to zero we could make the step size bigger with the risk of going over the zero loss point or we can do more training iterations

Expected result:

[1.0, -1.0, -1.0, 1.0]
---------------------------------------

[Value(data=-0.24400936649583502), Value(data=-0.739663299092658), Value(data=-0.21871892658341632), Value(data=-0.025416361134800725)]
0 {'loss': 3.2772133311305924}
[Value(data=0.8322184977334336), Value(data=-0.3353220845100484), Value(data=0.3341790388394802), Value(data=0.783633287348134)]
1 {'loss': 2.296795625865308}
[Value(data=0.15801968722824927), Value(data=-0.8315572548017955), Value(data=-0.5300314627113144), Value(data=0.5061898403834525)]
2 {'loss': 1.2020227052869092}
[Value(data=0.8832532219089855), Value(data=-0.7274375413201406), Value(data=-0.0587105071435734), Value(data=0.8266276054380944)]
3 {'loss': 1.0040040006342805}
[Value(data=0.7454817956057705), Value(data=-0.8752411770477271), Value(data=-0.6390090343577011), Value(data=0.7329122928186784)]
4 {'loss': 0.28199460087523415}
[Value(data=0.7720799486677041), Value(data=-0.8864065753652879), Value(data=-0.6710445768678128), Value

[Value(data=0.9580837237336022), Value(data=-0.9815934965908375), Value(data=-0.9487642623862351), Value(data=0.9718609832684983)]
142 {'loss': 0.005512678655234787}
[Value(data=0.958238717532177), Value(data=-0.9816623304028895), Value(data=-0.9489624727019451), Value(data=0.9719726497565923)]
143 {'loss': 0.005470636393976433}
[Value(data=0.9583920529565114), Value(data=-0.9817304158048604), Value(data=-0.9491585100083086), Value(data=0.9720830622646784)]
144 {'loss': 0.005429211480930117}
[Value(data=0.9585437590944962), Value(data=-0.9817977661157545), Value(data=-0.949352413111249), Value(data=0.9721922436571909)]
145 {'loss': 0.005388390598866566}
[Value(data=0.9586938643284154), Value(data=-0.9818643943271559), Value(data=-0.9495442198611914), Value(data=0.9723002162250217)]
146 {'loss': 0.005348160807836589}
[Value(data=0.9588423963567265), Value(data=-0.9819303131134627), Value(data=-0.9497339671829487), Value(data=0.9724070017037654)]
147 {'loss': 0.005308509531975186}
[Value